In [1]:
# in a terminal run
# > USER_PATH=/home/krause/userdata/ make run-redis NS=train
# > USER_PATH=/home/krause/userdata/ make run-redis NS=test
# to allow access to the train and test namespaces

In [2]:
import os
import sys
import json
import pandas as pd
import numpy as np
from typing import Literal, TypedDict

In [3]:
sys.path.append("..")
os.environ["USER_PATH"] = "/home/krause/userdata/"
MODEL_OUTPUT_BASE = "/mnt/d/workspace/clotho/notebooks"
MODEL_OUTPUT_CP = os.path.join(MODEL_OUTPUT_BASE, "checkpoints")

In [4]:
from misc.redis import set_redis_slow_mode
from misc.util import highest_number
from misc.io import open_write
from model.datagenerator import create_train_test
from model.transformer_embed import (
    get_epoch_and_load,
    limit_epoch_data,
    limit_epoch_data,
    get_model_filename,
)
from system.namespace.store import get_namespace

In [5]:
import torch

is_cuda = torch.cuda.is_available()
is_cuda

True

In [6]:
set_redis_slow_mode("never")
ns_test = get_namespace("test")
ns_train = get_namespace("train")
now = pd.Timestamp("2022-12-17", tz="UTC")
train_plan = [
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 10,
        "last_epoch": None,
        "weight": 50,
    },
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 10,
        "last_epoch": None,
        "weight": 50,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 40,
    },
     {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 5,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "path", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 5,
        "last_epoch": None,
        "weight": 40,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": True,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": None,
        "last_epoch": None,
        "weight": 40,
    },
    {
        "left": {"mode": "valid", "flip_pc": 1.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "first_epoch": 15,
        "last_epoch": None,
        "weight": 50,
    }
]
eval_plan = [
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 40,
    },
    {
        "left": {"mode": "random", "flip_pc": 0.0},
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 60,
    },
    {
        "left": None,
        "right": {"mode": "valid", "flip_pc": 0.0},
        "min_text_length": None,
        "skip_weak": False,
        "skip_topics": True,
        "flip_lr": 0.5,
        "weight": 40,
    },
]
ttgen = create_train_test(
    train_ns=ns_train,
    train_validation_ns=ns_train,
    test_ns=ns_test,
    test_validation_ns=ns_test,
    train_learning_plan=train_plan,
    train_val_learning_plan=eval_plan,
    test_learning_plan=eval_plan,
    test_val_learning_plan=eval_plan,
    batch_size=4 if is_cuda else 8,
    epoch_batches=5000 if is_cuda else 500,
    train_val_size=10000 if is_cuda else 1000,
    test_size=10000 if is_cuda else 1000,
    test_val_size=10000 if is_cuda else 1000,
    compute_batch_size=100 if is_cuda else 100,
    now=now)

In [7]:
import torch.nn as nn
from torch.optim import AdamW
from transformers import DistilBertTokenizer, DistilBertModel

In [8]:
device = torch.device("cuda") if is_cuda else torch.device("cpu")
device

device(type='cuda')

In [17]:
ProviderRole = Literal["child", "parent"]

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
EMBED_SIZE = 768

TokenizedInput = TypedDict('TokenizedInput', {
    "input_ids": torch.Tensor,
    "attention_mask": torch.Tensor,
})


AggType = Literal["cls", "mean"]
AGG_CLS: AggType = "cls"
AGG_MEAN: AggType = "mean"


def tokens(texts: list[str]) -> TokenizedInput:
    res = tokenizer(texts.tolist(), return_tensors="pt", padding=True, truncation=True)
    return {k: v.to(device) for k, v in res.items()}


class Noise(nn.Module):
    def __init__(self, std: float = 1.0, p: float = 0.5) -> None:
        super().__init__()
        self._std = std
        self._p = p
        self._dhold = nn.Parameter(torch.Tensor([0.0]), requires_grad=False)

    def set_std(self, std: float) -> None:
        self._std = std

    def get_std(self) -> float:
        return self._std

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not self.training:
            return x
        prob = torch.rand(size=x.shape, device=self._dhold.device) < self._p
        gauss = torch.normal(
            mean=0.0, std=self._std, size=x.shape, device=self._dhold.device)
        return x + prob * gauss


class Model(nn.Module):
    def __init__(self, version: int) -> None:
        super().__init__()
        assert version >= 0
        self._bert_parent = DistilBertModel.from_pretrained(
            "distilbert-base-uncased")
        self._bert_child = DistilBertModel.from_pretrained(
            "distilbert-base-uncased")
        if version in (1, 3, 4, 6):
            self._pdense: nn.Sequential | None = nn.Sequential(
                nn.Linear(EMBED_SIZE, EMBED_SIZE),
                nn.Dropout(p=0.2),
                nn.ReLU(),
                nn.Linear(EMBED_SIZE, EMBED_SIZE))
            self._cdense: nn.Sequential | None = nn.Sequential(
                nn.Linear(EMBED_SIZE, EMBED_SIZE),
                nn.Dropout(p=0.2),
                nn.ReLU(),
                nn.Linear(EMBED_SIZE, EMBED_SIZE))
        else:
            self._pdense = None
            self._cdense = None
        if version < 4 or version > 5:
            self._noise = None
        else:
            self._noise = Noise(std=1.0, p=0.2)
        if version < 2 or version > 4:
            self._cos = None
        else:
            self._cos = torch.nn.CosineSimilarity()
        if version < 6:
            self._agg = AGG_CLS
        else:
            self._agg = AGG_MEAN
        self._version = version

    def set_epoch(self, epoch: int) -> None:
        noise = self._noise
        if noise is not None:
            noise.set_std(1 / (1.2 ** epoch))

    def get_version(self) -> int:
        return self._version

    def get_agg(self, lhs: torch.Tensor) -> torch.Tensor:
        if self._agg == AGG_CLS:
            return lhs[:, 0]
        if self._agg == AGG_MEAN:
            return torch.mean(lhs, dim=1)
        raise ValueError(f"unknown aggregation: {self._agg}")

    def get_parent_embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        outputs_parent = self._bert_parent(
            input_ids=input_ids, attention_mask=attention_mask)
        out = self.get_agg(outputs_parent.last_hidden_state)
        if self._pdense is not None:
            out = self._pdense(out)
        if self._noise is not None:
            out = self._noise(out)
        return out

    def get_child_embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        outputs_child = self._bert_child(
            input_ids=input_ids, attention_mask=attention_mask)
        out = self.get_agg(outputs_child.last_hidden_state)
        if self._cdense is not None:
            out = self._cdense(out)
        if self._noise is not None:
            out = self._noise(out)
        return out

    def forward(self, x: dict[ProviderRole, TokenizedInput]) -> torch.Tensor:
        parent_cls = self.get_parent_embed(
            input_ids=x["parent"]["input_ids"],
            attention_mask=x["parent"]["attention_mask"])
        child_cls = self.get_child_embed(
            input_ids=x["child"]["input_ids"],
            attention_mask=x["child"]["attention_mask"])
        if self._cos is not None:
            return self._cos(parent_cls, child_cls).reshape([-1, 1])
        batch_size = parent_cls.shape[0]
        return torch.bmm(
            parent_cls.reshape([batch_size, 1, -1]),
            child_cls.reshape([batch_size, -1, 1])).reshape([-1, 1])


class BaselineModel(nn.Module):
    def __init__(self, version: int) -> None:
        super().__init__()
        assert version < 0
        self._bert = DistilBertModel.from_pretrained(
            "distilbert-base-uncased")
        if version == -2:
            self._agg = AGG_CLS
        else:
            self._agg = AGG_MEAN
        self._version = version

    def set_epoch(self, epoch: int) -> None:
        pass

    def get_version(self) -> int:
        return self._version

    def get_agg(self, lhs: torch.Tensor) -> torch.Tensor:
        if self._agg == AGG_CLS:
            return lhs[:, 0]
        if self._agg == AGG_MEAN:
            return torch.mean(lhs, dim=1)
        raise ValueError(f"unknown aggregation: {self._agg}")

    def _embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        outputs = self._bert(
            input_ids=input_ids, attention_mask=attention_mask)
        return self.get_agg(outputs.last_hidden_state)

    def get_parent_embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        return self._embed(input_ids, attention_mask)

    def get_child_embed(
            self,
            input_ids: torch.Tensor,
            attention_mask: torch.Tensor) -> torch.Tensor:
        return self._embed(input_ids, attention_mask)

    def forward(self, x: dict[ProviderRole, TokenizedInput]) -> torch.Tensor:
        parent_cls = self.get_parent_embed(
            input_ids=x["parent"]["input_ids"],
            attention_mask=x["parent"]["attention_mask"])
        child_cls = self.get_child_embed(
            input_ids=x["child"]["input_ids"],
            attention_mask=x["child"]["attention_mask"])
        batch_size = parent_cls.shape[0]
        return torch.bmm(
            parent_cls.reshape([batch_size, 1, -1]),
            child_cls.reshape([batch_size, -1, 1])).reshape([-1, 1])


EitherModel = Model | BaselineModel


class TrainingHarness(nn.Module):
    def __init__(self, model: EitherModel) -> None:
        super().__init__()
        self._model = model
        self._softmax = nn.Softmax(dim=1)
        self._loss = nn.BCELoss()

    def get_version(self) -> int:
        return self._model.get_version()

    def forward(
            self,
            left: TokenizedInput,
            right: TokenizedInput,
            labels: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        out_left = self._model(left)
        out_right = self._model(right)
        preds = self._softmax(torch.hstack((out_left, out_right)))
        return preds, self._loss(preds, labels)

In [18]:
from transformers import get_scheduler
# from tqdm.notebook import tqdm
from tqdm.auto import tqdm
import evaluate
import time


def create_model(version: int) -> EitherModel:
    return Model(version) if version >= 0 else BaselineModel(version)


def compute(harness, df):
    plefts = tokens(df["parent_left"])
    clefts = tokens(df["child_left"])
    prights = tokens(df["parent_right"])
    crights = tokens(df["child_right"])
    labels = torch.tensor(
        [~df["correct_is_right"], df["correct_is_right"]],
        dtype=torch.float32).T.to(device)
    return harness(
        left={"parent": plefts, "child": clefts},
        right={"parent": prights, "child": crights},
        labels=labels)


def run_training(num_epochs, version, force_restart):
    model = create_model(version)
    model.to(device)
    harness = TrainingHarness(model)
    harness.to(device)

    mprev, epoch_offset = get_epoch_and_load(
        harness,
        MODEL_OUTPUT_CP,
        ftype="harness",
        is_cuda=is_cuda,
        device=device,
        force_restart=force_restart)

    optimizer = AdamW(harness.parameters(), lr=5e-5)
    print(mprev, epoch_offset)
    
    num_epochs -= epoch_offset
    if num_epochs <= 0:
        print("already computed all epochs. nothing to do!")
        return model, harness, optimizer
    
    num_training_steps = num_epochs * ttgen.get_epoch_train_size()
    warmup = 10000 if is_cuda else 10
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=warmup,
        num_training_steps=num_training_steps - warmup)
    ttgen.set_epoch(epoch_offset)
    
    log_csv = get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="val_log",
        epoch=None,
        ext=".csv")
    columns = [
        "epoch",
        "train_acc",
        "train_loss",
        "train_val_acc",
        "train_val_loss",
        "test_acc",
        "test_loss",
        "time",
        "version",
        "fname",
    ]
    if not os.path.exists(log_csv):
        pd.DataFrame([], columns=columns).to_csv(
            log_csv, header=True, mode="w", columns=columns)

    for _ in range(num_epochs):
        epoch = ttgen.get_epoch()
        print(f"epoch {epoch} version: {harness.get_version()}")
        real_time = time.monotonic()

        model.train()
        harness.train()
        model.set_epoch(epoch)
        metric_train = evaluate.load("accuracy")
        train_loss = []
        first = True
        with tqdm(desc="train", total=ttgen.get_epoch_train_size()) as progress_bar:
            for train_df in ttgen.train_dfs():
                preds, loss = compute(harness, train_df)
                train_loss.append(loss.item())
                loss.backward()

                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(train_df.shape[0])

                predictions = torch.argmax(preds, dim=-1)
                metric_train.add_batch(
                    predictions=predictions,
                    references=train_df["correct_is_right"].astype(int))
                if first:
                    # display(train_df)
                    first = False

        model_fname = get_model_filename(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="harness",
            epoch=epoch)
        torch.save(harness.state_dict(), model_fname)

        model.eval()
        harness.eval()
        with torch.no_grad():
            metric_val_train = evaluate.load("accuracy")
            train_val_loss = []
            with tqdm(desc="train val", total=ttgen.get_epoch_train_validation_size()) as progress_bar:
                for train_validation_df in ttgen.train_validation_dfs():
                    preds, loss = compute(harness, train_validation_df)
                    train_val_loss.append(loss.item())
                    predictions = torch.argmax(preds, dim=-1)
                    metric_val_train.add_batch(
                        predictions=predictions,
                        references=train_validation_df["correct_is_right"].astype(int))
                    progress_bar.update(train_validation_df.shape[0])

            metric_test = evaluate.load("accuracy")
            test_loss = []
            with tqdm(desc="test", total=ttgen.get_epoch_test_size()) as progress_bar:
                for test_df in ttgen.test_dfs():
                    preds, loss = compute(harness, test_df)
                    test_loss.append(loss.item())
                    predictions = torch.argmax(preds, dim=-1)
                    metric_test.add_batch(
                        predictions=predictions,
                        references=test_df["correct_is_right"].astype(int))
                    progress_bar.update(test_df.shape[0])
            stats = {
                "epoch": int(epoch),
                "train_acc": float(metric_train.compute()['accuracy']),
                "train_loss": float(np.mean(train_loss)),
                "train_val_acc": float(metric_val_train.compute()['accuracy']),
                "train_val_loss": float(np.mean(train_val_loss)),
                "test_acc": float(metric_test.compute()['accuracy']),
                "test_loss": float(np.mean(test_loss)),
                "time": 0.0,
                "version": harness.get_version(),
                "fname": model_fname,
            }

        print(f"train: {stats['train_acc']} loss: {stats['train_loss']}")
        print(f"train val: {stats['train_val_acc']} loss: {stats['train_val_loss']}")
        print(f"test: {stats['test_acc']} loss: {stats['test_loss']}")
        ttgen.advance_epoch()
        stats["time"] = float((time.monotonic() - real_time) / 60.0)
        print(f"epoch time: {stats['time']:.2f}min")
        stats_fn = get_model_filename(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="stats",
            epoch=epoch,
            ext=".json")
        with open_write(stats_fn, text=True) as fout:
            print(json.dumps(stats, indent=2, sort_keys=True), file=fout)
        stats_df = pd.DataFrame(
            {key: [val] for key, val in stats.items()},
            columns=columns)
        stats_df.to_csv(
            log_csv, header=False, mode="a")
            
        limit_epoch_data(
            harness,
            MODEL_OUTPUT_CP,
            is_cuda=is_cuda,
            ftype="stats",
            ext=".json",
            count=5)
    return model, harness, optimizer

In [19]:
def save_model(model, harness, optimizer):
    torch.save(model.state_dict(), get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="model",
        epoch=None))
    torch.save(harness.state_dict(), get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="harness",
        epoch=None))
    torch.save(optimizer.state_dict(), get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="optimizer",
        epoch=None))

In [20]:
def validation(model, harness):
    ttgen.reset()
    model.eval()
    harness.eval()
    dfs = []
    with torch.no_grad():
        metric_val_test = evaluate.load("accuracy")
        test_val_loss = []
        with tqdm(desc="test val", total=ttgen.get_epoch_test_validation_size()) as progress_bar:
            for test_val_df in ttgen.test_validation_dfs():
                preds, loss = compute(harness, test_val_df)
                test_val_loss.append(loss.item())
                predictions = torch.argmax(preds, dim=-1)
                metric_val_test.add_batch(
                    predictions=predictions,
                    references=test_val_df["correct_is_right"].astype(int))
                cur_df = test_val_df.copy()
                cur_df["logit_left"] = preds[:, 0].cpu()
                cur_df["logit_right"] = preds[:, 1].cpu()
                cur_df["preds"] = predictions.cpu()
                cur_df["truth"] = test_val_df["correct_is_right"].astype(int)
                dfs.append(cur_df)
                progress_bar.update(test_val_df.shape[0])
    print(f"test val: {metric_val_test.compute()} loss: {np.mean(test_val_loss)}")
    validation_df = pd.concat(dfs)
    validation_df.to_csv(get_model_filename(
        harness,
        MODEL_OUTPUT_BASE,
        is_cuda=is_cuda,
        ftype="validation",
        epoch=None,
        ext=".csv"))
    print("correct")
    display(validation_df[validation_df["preds"] == validation_df["truth"]].head())
    print("incorrect")
    display(validation_df[validation_df["preds"] != validation_df["truth"]].head())

In [21]:
def embeds(model):
    ttgen.reset()
    model.eval()
    with torch.no_grad():
        count = 0
        for test_val_df in ttgen.test_validation_dfs():
            plefts = tokens(test_val_df["parent_left"])
            clefts = tokens(test_val_df["child_left"])
            prights = tokens(test_val_df["parent_right"])
            crights = tokens(test_val_df["child_right"])
            display(model.get_child_embed(
                clefts["input_ids"],
                clefts["attention_mask"]).cpu().numpy())
            display(model.get_child_embed(
                crights["input_ids"],
                crights["attention_mask"]).cpu().numpy())
            count += 1
            if count >= 5:
                break

In [22]:
def full_run(*, num_epochs, version, force_restart):
    model, harness, optimizer = run_training(
        num_epochs, version, force_restart)
    save_model(model, harness, optimizer)
    validation(model, harness)
    embeds(model)

In [23]:
# for version in range(8):
#     full_run(num_epochs=30, version=version, force_restart=False)

In [24]:
full_run(num_epochs=30, version=-1, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


None 0
epoch 0 version: -1


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[0.04451372, 0.26753968, 0.14872947, ..., 0.2072586 , 0.05033529,
        0.12699212],
       [0.10094371, 0.27681297, 0.15124492, ..., 0.20351683, 0.08930163,
        0.04914492],
       [0.101463  , 0.24599555, 0.20501748, ..., 0.21281101, 0.08309069,
        0.15020776],
       [0.07212768, 0.24315128, 0.17190704, ..., 0.20010717, 0.07074664,
        0.1240553 ]], dtype=float32)

array([[0.04088778, 0.22220714, 0.18750073, ..., 0.2110226 , 0.10171761,
        0.09561012],
       [0.07189051, 0.25865635, 0.19133043, ..., 0.21333188, 0.11329992,
        0.09859306],
       [0.10516229, 0.26286185, 0.1343041 , ..., 0.24138553, 0.03868827,
        0.16120382],
       [0.12646155, 0.2659252 , 0.15781476, ..., 0.21670517, 0.083362  ,
        0.12453119]], dtype=float32)

array([[0.10278735, 0.261392  , 0.19462354, ..., 0.19629607, 0.10508237,
        0.09148873],
       [0.01169097, 0.21965125, 0.18232442, ..., 0.16835526, 0.07425379,
        0.11272497],
       [0.13274528, 0.27420542, 0.17499395, ..., 0.23856232, 0.09475509,
        0.11544965],
       [0.10445559, 0.2716134 , 0.19586547, ..., 0.23201224, 0.11532611,
        0.10299481]], dtype=float32)

array([[0.04515429, 0.20944549, 0.16893414, ..., 0.23550043, 0.0762472 ,
        0.1522683 ],
       [0.1391955 , 0.27980894, 0.18513805, ..., 0.21002716, 0.085454  ,
        0.10228127],
       [0.10004909, 0.23407501, 0.1893128 , ..., 0.23639928, 0.08314568,
        0.13458422],
       [0.07630298, 0.2505793 , 0.17952417, ..., 0.23999168, 0.1128565 ,
        0.08795626]], dtype=float32)

array([[0.05608737, 0.25853637, 0.1607169 , ..., 0.24087219, 0.1031827 ,
        0.12489446],
       [0.03183381, 0.22652684, 0.17239109, ..., 0.21221349, 0.09734751,
        0.1003384 ],
       [0.06282149, 0.23636988, 0.16687582, ..., 0.21612741, 0.09005637,
        0.11346034],
       [0.10946818, 0.27028346, 0.18167287, ..., 0.23133129, 0.08723096,
        0.15608232]], dtype=float32)

In [25]:
full_run(num_epochs=30, version=7, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- T

('harness_v7_lg_11.pkl', 11) 12
epoch 12 version: 7


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

array([[ 0.6303296 ,  0.0297453 , -0.86188084, ...,  0.7713893 ,
        -0.79193443,  0.07404995],
       [ 0.6306972 ,  0.02964074, -0.86285675, ...,  0.7731063 ,
        -0.79093444,  0.07446814],
       [ 0.6321327 ,  0.02973243, -0.8655982 , ...,  0.77833074,
        -0.7878914 ,  0.07544802],
       [ 0.6319434 ,  0.029688  , -0.8652916 , ...,  0.7777224 ,
        -0.78824276,  0.07535724]], dtype=float32)

array([[ 0.63027716,  0.02976605, -0.8617264 , ...,  0.7711252 ,
        -0.79208803,  0.07398187],
       [ 0.6296586 ,  0.03013363, -0.8595165 , ...,  0.7675371 ,
        -0.79417866,  0.07297672],
       [ 0.6317902 ,  0.02965796, -0.8650355 , ...,  0.7772183 ,
        -0.7885351 ,  0.07527722],
       [ 0.631381  ,  0.02960585, -0.86430544, ...,  0.77580106,
        -0.78935987,  0.07503057]], dtype=float32)

array([[ 0.6320891 ,  0.02972127, -0.8655277 , ...,  0.7781906 ,
        -0.78797203,  0.07542776],
       [ 0.63083607,  0.02961862, -0.86318225, ...,  0.7736973 ,
        -0.7905892 ,  0.07460173],
       [ 0.6318703 ,  0.02967309, -0.86517036, ...,  0.7774838 ,
        -0.78838146,  0.07531975],
       [ 0.6298625 ,  0.02998879, -0.86032414, ...,  0.76880944,
        -0.79343504,  0.07334781]], dtype=float32)

array([[ 0.63005626,  0.02987203, -0.86101973, ...,  0.7699399 ,
        -0.79277706,  0.07366505],
       [ 0.63185006,  0.02966933, -0.86513627, ...,  0.7774174 ,
        -0.7884205 ,  0.0753087 ],
       [ 0.63085365,  0.02961643, -0.8632226 , ...,  0.77377075,
        -0.7905463 ,  0.07461797],
       [ 0.62971985,  0.030088  , -0.8597668 , ...,  0.76792705,
        -0.7939502 ,  0.07309181]], dtype=float32)

array([[ 0.63005203,  0.02987439, -0.8610049 , ...,  0.76991546,
        -0.7927915 ,  0.07365836],
       [ 0.6317439 ,  0.02964981, -0.8649564 , ...,  0.7770627 ,
        -0.7886252 ,  0.07525201],
       [ 0.63119847,  0.02959836, -0.8639529 , ...,  0.77513003,
        -0.789752  ,  0.07490178],
       [ 0.63110656,  0.0295988 , -0.8637668 , ...,  0.77477986,
        -0.7899567 ,  0.07483163]], dtype=float32)

In [26]:
full_run(num_epochs=30, version=0, force_restart=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- T

None 0
epoch 0 version: 0


train:   0%|          | 0/20000 [00:00<?, ?it/s]

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

...

train val:   0%|          | 0/10000 [00:00<?, ?it/s]

test:   0%|          | 0/10000 [00:00<?, ?it/s]

train: 0.83565 loss: 0.3272408008855768
train val: 0.7 loss: 0.5112185107074678
test: 0.6345 loss: 0.6393793481111526
epoch time: 23.73min
removing /mnt/d/workspace/clotho/notebooks/checkpoints/stats_v0_lg_3.json
removing /mnt/d/workspace/clotho/notebooks/checkpoints/harness_v0_lg_3.pkl
best model: /mnt/d/workspace/clotho/notebooks/checkpoints/harness_v0_lg_6.pkl
best train: 0.814
best train val: 0.7149
best test: 0.632
epoch 10 version: 0


train:   0%|          | 0/20000 [00:00<?, ?it/s]

OSError: [Errno 28] No space left on device

In [ ]:
full_run(num_epochs=30, version=5, force_restart=False)

In [ ]:
full_run(num_epochs=60, version=-1, force_restart=False)

In [ ]:
full_run(num_epochs=60, version=7, force_restart=False)

In [ ]:
full_run(num_epochs=60, version=0, force_restart=False)

In [ ]:
full_run(num_epochs=60, version=5, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=-1, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=7, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=0, force_restart=False)

In [ ]:
full_run(num_epochs=90, version=5, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=-1, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=7, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=0, force_restart=False)

In [ ]:
full_run(num_epochs=120, version=5, force_restart=False)